# Segmenting and Clustering Neighborhoods in Toronto
This notebook is used for the Week 3 Assignment

## Import libraries

In [1]:
import re
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

## Scrape basic neighborhood information

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url).content

In [3]:
soup = BeautifulSoup(result, "html.parser")
code_collections = soup.find('tbody').find_all('tr')

In [4]:
def parse_neighborhood(content):
    neighborhood = ""
    # use regular expression to extract neighborhood in the bracket
    for index, part in enumerate(re.findall("(?<=\()[^\)]+(?=\))", content)):
        if index == 0:
            neighborhood = ",".join(part.split("/"))
        else:
            neighborhood = neighborhood + "," + ",".join(part.split("/"))
    return neighborhood

In [5]:
toronto_data = pd.DataFrame(columns = ["PostalCode", "Borough", "Neighborhood"])
for raw in code_collections:
    for entry in raw.find_all('td'):
        # extract the information if the code is assigned
        if not 'Not assigned'in entry.get_text():
            content = entry.find('span')
            borough_element = content.find('br').previous_sibling
            try:
                # when the element is a hyperlink 
                borough = borough_element.get_text()
            except:
                # when the element is just plan text
                borough = borough_element
            code = entry.find_all('b')[0].get_text()
            neighborhood = parse_neighborhood(content.get_text())
            toronto_data = toronto_data.append(pd.Series({"PostalCode":code, "Borough" :borough, "Neighborhood" :neighborhood}), ignore_index = True)
        # skip if it's not assigned
        else:
            continue

In [6]:
toronto_data.sort_values(["PostalCode"])
toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [7]:
toronto_data.shape

(103, 3)